In [2]:
import numpy as np  # Mengimpor pustaka NumPy dengan alias np untuk digunakan nanti.
import pandas as pd  # Mengimpor pustaka Pandas dengan alias pd untuk digunakan nanti.

df = pd.read_csv('../Pertemuan-04/data/spam.csv', encoding='latin-1')  # Membaca file 'spam.csv' ke dalam sebuah DataFrame menggunakan Pandas. Spesifikasi encoding 'latin-1' diperlukan karena data tidak menggunakan format UTF-8.

df.head()  # Menampilkan baris pertama dari DataFrame 'df' untuk melihat data awalnya.


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
# Menggunakan metode iloc untuk menghapus tiga kolom terakhir dari DataFrame 'df' dengan menggunakan slicing [:,2:]. Axis=1 menunjukkan bahwa penghapusan dilakukan pada kolom, bukan baris.

df = df.drop(df.iloc[:,2:], axis=1)

# Menampilkan lima baris pertama dari DataFrame 'df' setelah penghapusan kolom.
df.head()


,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
# Mendefinisikan dictionary 'new_cols' yang akan digunakan untuk merename kolom 'v1' menjadi 'Labels' dan kolom 'v2' menjadi 'SMS'.
new_cols = {
    'v1': 'Labels',
    'v2': 'SMS'
}

# Menggunakan metode 'rename' dari Pandas untuk merename nama kolom sesuai dengan dictionary 'new_cols'.
df = df.rename(columns=new_cols)

# Menampilkan lima baris pertama dari DataFrame 'df' setelah proses rename kolom.
df.head()


,Labels,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
# Cek jumlah data per kelas (Labels) dan menampilkannya.
print(df['Labels'].value_counts())
print('\n')

# Cek kelengkapan data seperti jumlah entri, tipe data, dan penggunaan memori.
print(df.info())
print('\n')

# Cek statistik deskriptif untuk kolom-kolom numerik dalam DataFrame.
print(df.describe())


ham     4825
spam     747
Name: Labels, dtype: int64


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Labels  5572 non-null   object
 1   SMS     5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB
None


       Labels                     SMS
count    5572                    5572
unique      2                    5169
top       ham  Sorry, I'll call later
freq     4825                      30


In [6]:
# Mendefinisikan dictionary 'new_labels' yang akan digunakan untuk mengkodekan label 'spam' menjadi 1 dan label 'ham' menjadi 0.
new_labels = {
    'spam': 1,
    'ham': 0
}

# Menggunakan metode 'map' untuk mengganti nilai dalam kolom 'Labels' sesuai dengan dictionary 'new_labels'.
df['Labels'] = df['Labels'].map(new_labels)

# Menampilkan lima baris pertama dari DataFrame 'df' setelah proses pengkodean label.
df.head()


,Labels,SMS
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
X = df['SMS'].values  # Menyimpan nilai dari kolom 'SMS' dalam variabel X.
y = df['Labels'].values  # Menyimpan nilai dari kolom 'Labels' dalam variabel y.


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# Mengimpor pustaka yang diperlukan: train_test_split dari sklearn untuk pemisahan data, dan CountVectorizer untuk pengolahan teks.

# Memisahkan data menjadi data training dan testing dengan rasio 80:20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)

# Inisiasi objek CountVectorizer untuk mengubah teks menjadi vektor
bow = CountVectorizer()

# Fitting dan transformasi data X_train menggunakan CountVectorizer
X_train = bow.fit_transform(X_train)

# Transformasi data X_test dengan CountVectorizer
# Penjelasan mengapa hanya transform: kita tidak ingin model mengetahui parameter yang digunakan oleh CountVectorizer pada data X_train,
# sehingga data testing tetap menjadi data yang tidak dikenali oleh model.
X_test = bow.transform(X_test)


In [15]:
print(len(bow.vocabulary_))  # Menampilkan jumlah fitur yang dihasilkan oleh CountVectorizer.
print(f'Dimensi data: {X_train.shape}')  # Menampilkan dimensi data pelatihan (jumlah baris, jumlah fitur).


7727
Dimensi data: (4457, 7727)


In [16]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Mengimpor pustaka yang diperlukan: MultinomialNB dari sklearn untuk model Naive Bayes dan accuracy_score untuk mengukur akurasi.

# Inisiasi model Naive Bayes Multinomial
mnb = MultinomialNB()

# Melatih (fit) model menggunakan data pelatihan
mnb.fit(X_train, y_train)

# Melakukan prediksi menggunakan data pelatihan
y_pred_train = mnb.predict(X_train)

# Evaluasi akurasi pada data pelatihan
acc_train = accuracy_score(y_train, y_pred_train)

# Melakukan prediksi menggunakan data pengujian
y_pred_test = mnb.predict(X_test)

# Evaluasi akurasi pada data pengujian
acc_test = accuracy_score(y_test, y_pred_test)

# Mencetak hasil evaluasi akurasi pada data pelatihan dan pengujian
print(f'Hasil akurasi data train: {acc_train}')
print(f'Hasil akurasi data test: {acc_test}')


Hasil akurasi data train: 0.9946152120260264
Hasil akurasi data test: 0.9775784753363229
